### Test notebook for connecting to AWS buckets and designing tables/ETL

In [ ]:
import configparser
import psycopg2
import pandas

### Check datawarehouse config file

In [ ]:
config = configparser.ConfigParser()
config.read('dwh.cfg')